# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [31]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Processing...
Done!


Then, we define the model, object function and optimizer that we use to classify.

In [5]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1,16,kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(nn.Conv2d(16,64,kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2))
        self.dense = nn.Sequential(nn.Linear(7*7*64,1024),
                                   nn.ReLU(),
                                   nn.Dropout(p=0.5),
                                   nn.Linear(1024, 10))
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1,7*7*64)
        x = self.dense(x)
        return x

    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [32]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        
        optimizer.zero_grad()
        predictions = model(images)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
model.eval()
correct = 0
total = 0
for images, labels in train_loader:
    output = model(images)
    _, predicted = torch.max(output,1)
    total += labels.size(0)
    correct +=(predicted == labels).sum()
training_accuracy = 100.*correct.data/total
    
correct = 0
total = 0
for images, labels in test_loader:
    output = model(images)
    _, predicted = torch.max(output,1)
    total += labels.size(0)
    correct +=(predicted == labels).sum()

testing_accuracy = 100.*correct.data/total

100%|██████████| 468/468 [00:48<00:00,  9.57it/s]


#### Q5:
Please print the training and testing accuracy.

In [33]:
print('Training accuracy: %0.2f%%' % (training_accuracy))
print('Testing accuracy: %0.2f%%' % (testing_accuracy))

Training accuracy: 99.85%
Testing accuracy: 99.05%
